In [17]:
from radiomics import featureextractor  # Make sure to import the required modules

def setup_extractor():
    params = {
        'label': 1,  # Define which label value should be used for feature extraction
        'binWidth': 25,  # Set bin width for histogram calculations
        'resampledPixelSpacing': None,  # No resampling by default
        'interpolator': 'sitkBSpline',  # Interpolation method
        'enableCExtensions': True
    }
    extractor = featureextractor.RadiomicsFeatureExtractor(**params)
    extractor.enableAllFeatures()  # Enable all available features
    return extractor

In [8]:
import SimpleITK as sitk
import numpy as np

def convert_mask_values(mask_image_path, new_value=1):
    mask = sitk.ReadImage(mask_image_path)
    mask_array = sitk.GetArrayFromImage(mask)
    mask_array[mask_array == 255] = new_value  # Convert 255 to 1
    new_mask = sitk.GetImageFromArray(mask_array)
    new_mask.CopyInformation(mask)
    return new_mask


In [15]:
import os
import csv
from radiomics import featureextractor
import SimpleITK as sitk

def extract_radiomics_features(image_dir, mask_dir, output_csv):
    # Configure the radiomics feature extractor
    extractor = setup_extractor()  # Set up the feature extractor with the specified label
    # Prepare to write output to CSV
    with open(output_csv, 'w', newline='') as f:
        writer = None  # We will initialize the writer after we know the feature names

        # Walk through the image directory structure
        for subdir, dirs, files in os.walk(image_dir):
            for file in files:
                if file.endswith('.png'):  # Ensure we are processing .png files
                    image_path = os.path.join(subdir, file)
                    # Construct the corresponding mask path
                    mask_filename = file[:-4] + '_mask.png'  # Remove '.png' and add '_mask.png'
                    mask_path = os.path.join(mask_dir, os.path.basename(subdir), mask_filename)
                    
                    if os.path.exists(mask_path):
                        # Read the image and mask
                        image = sitk.ReadImage(image_path)
                        mask = sitk.ReadImage(mask_path)
                        
                        # Extract features
                        features = extractor.execute(image, mask)

                        # Initialize CSV writer with header once
                        if writer is None:
                            header = ['Image', 'Mask'] + list(features.keys())
                            writer = csv.writer(f)
                            writer.writerow(header)

                        # Collect feature values
                        feature_values = [features[key] for key in features.keys()]
                        
                        # Write to CSV
                        writer.writerow([image_path, mask_path] + feature_values)
    print("Feature extraction complete. Output saved to:", output_csv)

# Specify directories and output file
image_dir = '/home/salma.hassan/AI702/MaskAttention/data/images/train'
mask_dir = '/home/salma.hassan/AI702/MaskAttention/data/masks/train_mask'
output_csv = '/home/salma.hassan/AI702/MaskAttention/radiomics/features.csv'

extract_radiomics_features(image_dir, mask_dir, output_csv)


RuntimeError: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: /tmp/SimpleITK/Code/Common/include/sitkDualMemberFunctionFactory.hxx:190:
sitk::ERROR: Pixel type: vector of 8-bit unsigned integer is not supported in 2D byN3itk6simple26LabelStatisticsImageFilterE

In [18]:
import SimpleITK as sitk

def read_and_convert_image(image_path):
    """ Read an image and convert it to grayscale if it is not already. """
    image = sitk.ReadImage(image_path)
    if image.GetNumberOfComponentsPerPixel() > 1:
        image = sitk.VectorMagnitude(image)
    return image

def read_and_prepare_mask(mask_path, label_value=1):
    """ Read a mask image, ensuring it is single-channel and convert it if necessary. """
    mask = sitk.ReadImage(mask_path, sitk.sitkUInt8)  # Force reading as single channel
    mask_array = sitk.GetArrayFromImage(mask)
    # Set the mask to have only the specified label value
    mask_array[mask_array != 0] = label_value
    new_mask = sitk.GetImageFromArray(mask_array)
    new_mask.CopyInformation(mask)
    return new_mask

# Usage of these functions in your feature extraction process:
def extract_radiomics_features(image_dir, mask_dir, output_csv, label_value=1):
    extractor = setup_extractor()  # Assuming setup_extractor is defined as before
    # Rest of the setup for CSV and walking directories...

    for subdir, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith('.png'):
                image_path = os.path.join(subdir, file)
                mask_filename = file[:-4] + '_mask.png'
                mask_path = os.path.join(mask_dir, os.path.basename(subdir), mask_filename)
                
                if os.path.exists(mask_path):
                    image = read_and_convert_image(image_path)
                    mask = read_and_prepare_mask(mask_path, label_value=label_value)

                    # Ensure the mask is the correct type and only contains the label value
                    features = extractor.execute(image, mask)
                    # Handling CSV output...
                    # Initialize CSV writer with header once
                    if writer is None:
                        header = ['Image', 'Mask'] + list(features.keys())
                        writer = csv.writer(f)
                        writer.writerow(header)

                    # Collect feature values
                    feature_values = [features[key] for key in features.keys()]
                    
                    # Write to CSV
                    writer.writerow([image_path, mask_path] + feature_values)
print("Feature extraction complete. Output saved to:", output_csv)

# Specify directories and output file
image_dir = '/home/salma.hassan/AI702/MaskAttention/data/images/train'
mask_dir = '/home/salma.hassan/AI702/MaskAttention/data/masks/train_mask'
output_csv = '/home/salma.hassan/AI702/MaskAttention/radiomics/features.csv'

extract_radiomics_features(image_dir, mask_dir, output_csv)



Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input


Feature extraction complete. Output saved to: /home/salma.hassan/AI702/MaskAttention/radiomics/features.csv


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


UnboundLocalError: local variable 'writer' referenced before assignment

In [19]:
import os
import csv
from radiomics import featureextractor  # Assuming you have this function defined correctly elsewhere
import SimpleITK as sitk

# Functions for reading and converting images and masks
def read_and_convert_image(image_path):
    """ Read an image and convert it to grayscale if it is not already. """
    image = sitk.ReadImage(image_path)
    if image.GetNumberOfComponentsPerPixel() > 1:
        image = sitk.VectorMagnitude(image)
    return image

def read_and_prepare_mask(mask_path, label_value=1):
    """ Read a mask image, ensuring it is single-channel and convert it if necessary. """
    mask = sitk.ReadImage(mask_path, sitk.sitkUInt8)  # Force reading as single channel
    mask_array = sitk.GetArrayFromImage(mask)
    mask_array[mask_array != 0] = label_value
    new_mask = sitk.GetImageFromArray(mask_array)
    new_mask.CopyInformation(mask)
    return new_mask

# Setup extractor from another module if not included here
def setup_extractor():
    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableAllFeatures()
    return extractor

# Main function for feature extraction
def extract_radiomics_features(image_dir, mask_dir, output_csv, label_value=1):
    extractor = setup_extractor()
    # Initialize CSV file and writer outside the directory loop
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        # Initialize header flag
        header_written = False

        for subdir, dirs, files in os.walk(image_dir):
            for file in files:
                if file.endswith('.png'):
                    image_path = os.path.join(subdir, file)
                    mask_filename = file[:-4] + '_mask.png'
                    mask_path = os.path.join(mask_dir, os.path.basename(subdir), mask_filename)
                    
                    if os.path.exists(mask_path):
                        image = read_and_convert_image(image_path)
                        mask = read_and_prepare_mask(mask_path, label_value=label_value)
                        
                        features = extractor.execute(image, mask)

                        # Write header once based on the features keys
                        if not header_written:
                            header = ['Image', 'Mask'] + list(features.keys())
                            writer.writerow(header)
                            header_written = True

                        # Collect feature values and write to CSV
                        feature_values = [features[key] for key in features.keys()]
                        writer.writerow([image_path, mask_path] + feature_values)

        print("Feature extraction complete. Output saved to:", output_csv)

# Specify directories and output file
image_dir = '/home/salma.hassan/AI702/MaskAttention/data/images/train'
mask_dir = '/home/salma.hassan/AI702/MaskAttention/data/masks/train_mask'
output_csv = '/home/salma.hassan/AI702/MaskAttention/radiomics/features.csv'

extract_radiomics_features(image_dir, mask_dir, output_csv)

Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is 

ValueError: No labels found in this mask (i.e. nothing is segmented)!

In [21]:
import os
import csv
from radiomics import featureextractor  # Assuming you have this function defined correctly elsewhere
import SimpleITK as sitk

def read_and_convert_image(image_path):
    """ Read an image and convert it to grayscale if it is not already. """
    image = sitk.ReadImage(image_path)
    if image.GetNumberOfComponentsPerPixel() > 1:
        image = sitk.VectorMagnitude(image)
    return image

def read_and_prepare_mask(mask_path, label_value=1):
    """ Read a mask image, ensuring it is single-channel, and convert it if necessary. """
    mask = sitk.ReadImage(mask_path, sitk.sitkUInt8)  # Force reading as single channel
    mask_array = sitk.GetArrayFromImage(mask)
    # Set the mask to have only the specified label value
    mask_array[mask_array != 0] = label_value
    new_mask = sitk.GetImageFromArray(mask_array)
    new_mask.CopyInformation(mask)
    return new_mask

def extract_radiomics_features(image_dir, mask_dir, output_csv, label_value=1):
    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableAllFeatures()

    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        header_written = False

        for subdir, dirs, files in os.walk(image_dir):
            for file in files:
                if file.endswith('.png'):
                    image_path = os.path.join(subdir, file)
                    mask_filename = file[:-4] + '_mask.png'
                    mask_path = os.path.join(mask_dir, os.path.basename(subdir), mask_filename)

                    if os.path.exists(mask_path):
                        image = read_and_convert_image(image_path)
                        try:
                            mask = read_and_prepare_mask(mask_path, label_value=label_value)
                            features = extractor.execute(image, mask)

                            if not header_written:
                                header = ['Image', 'Mask'] + list(features.keys())
                                writer.writerow(header)
                                header_written = True

                            feature_values = [features[key] for key in features.keys()]
                            writer.writerow([image_path, mask_path] + feature_values)
                        except ValueError as e:
                            print(f"Skipping {mask_path}: {e}")
        print("Feature extraction complete. Output saved to:", output_csv)

# Specify directories and output file
image_dir = '/home/salma.hassan/AI702/MaskAttention/data/images/test'
mask_dir = '/home/salma.hassan/AI702/MaskAttention/data/masks/test_mask'
output_csv = '/home/salma.hassan/AI702/MaskAttention/radiomics/features_test.csv'

extract_radiomics_features(image_dir, mask_dir, output_csv)


Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is 

Feature extraction complete. Output saved to: /home/salma.hassan/AI702/MaskAttention/radiomics/features_test.csv
